In [1]:
# VGG Like Model

In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
np.random.seed(12345)
#%matplotlib inline

/home/pdeepsingh91/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
train_images_path = keras.utils.get_file('train-images-idx3-ubyte.gz', 'https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/data/fashion/train-images-idx3-ubyte.gz')
train_labels_path = keras.utils.get_file('train-labels-idx1-ubyte.gz', 'https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/data/fashion/train-labels-idx1-ubyte.gz')
test_images_path = keras.utils.get_file('t10k-images-idx3-ubyte.gz', 'https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/data/fashion/t10k-images-idx3-ubyte.gz')
test_labels_path = keras.utils.get_file('t10k-labels-idx1-ubyte.gz', 'https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/data/fashion/t10k-labels-idx1-ubyte.gz')


In [4]:
batch_size = 512

In [5]:
def load_mnist(images_path, labels_path):
    import os
    import gzip
    import numpy as np

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [6]:
X_train_orig, y_train_orig = load_mnist(train_images_path, train_labels_path)
X_test, y_test = load_mnist(test_images_path, test_labels_path)
X_train_orig = X_train_orig.astype('float32')
X_test = X_test.astype('float32')
X_train_orig /= 255
X_test /= 255

In [7]:
print(X_train_orig.shape)
print(y_train_orig.shape)
print(X_test.shape)
print(y_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train_orig, y_train_orig, test_size=0.2, random_state=12345)

In [9]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(48000, 784)
(48000,)
(12000, 784)
(12000,)


In [10]:
#plt.imshow(X_train[1, :].reshape((28, 28)))

In [11]:
img_rows = 28
img_cols = 28
input_shape = (img_rows, img_cols, 1)
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)

In [12]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)
def norm_input(x): return (x-mean_px)/std_px

In [13]:
cnn4 = Sequential([
    Lambda(norm_input, input_shape=(28,28, 1)),
    Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape),
    Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),


    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),


    Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),


    Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),

    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')])

In [14]:
cnn4.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

In [20]:
cnn4.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 333s 7ms/step - loss: 1.1948 - acc: 0.5359 - val_loss: 0.6064 - val_acc: 0.7661
Epoch 2/10
48000/48000 [==============================] - 329s 7ms/step - loss: 0.5193 - acc: 0.8076 - val_loss: 0.3892 - val_acc: 0.8524
Epoch 3/10
48000/48000 [==============================] - 327s 7ms/step - loss: 0.3910 - acc: 0.8591 - val_loss: 0.3851 - val_acc: 0.8639
Epoch 4/10
48000/48000 [==============================] - 328s 7ms/step - loss: 0.3275 - acc: 0.8832 - val_loss: 0.2750 - val_acc: 0.9040
Epoch 5/10
48000/48000 [==============================] - 330s 7ms/step - loss: 0.2841 - acc: 0.8987 - val_loss: 0.2660 - val_acc: 0.9030
Epoch 6/10
48000/48000 [==============================] - 331s 7ms/step - loss: 0.2609 - acc: 0.9064 - val_loss: 0.2437 - val_acc: 0.9128
Epoch 7/10
48000/48000 [==============================] - 330s 7ms/step - loss: 0.2258 - acc: 0.9192 - val_loss: 0.2557 - 

In [21]:
cnn4.optimizer.lr = 0.0001


In [22]:
cnn4.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 329s 7ms/step - loss: 0.1607 - acc: 0.9426 - val_loss: 0.2254 - val_acc: 0.9240
Epoch 2/10
48000/48000 [==============================] - 328s 7ms/step - loss: 0.1474 - acc: 0.9479 - val_loss: 0.2151 - val_acc: 0.9241
Epoch 3/10
48000/48000 [==============================] - 329s 7ms/step - loss: 0.1343 - acc: 0.9524 - val_loss: 0.2210 - val_acc: 0.9273
Epoch 4/10
48000/48000 [==============================] - 329s 7ms/step - loss: 0.1225 - acc: 0.9560 - val_loss: 0.2254 - val_acc: 0.9264
Epoch 5/10
48000/48000 [==============================] - 330s 7ms/step - loss: 0.1086 - acc: 0.9621 - val_loss: 0.2279 - val_acc: 0.9285
Epoch 6/10
48000/48000 [==============================] - 331s 7ms/step - loss: 0.1000 - acc: 0.9648 - val_loss: 0.2638 - val_acc: 0.9262
Epoch 7/10
48000/48000 [==============================] - 330s 7ms/step - loss: 0.1015 - acc: 0.9642 - val_loss: 0.2462 - 

In [23]:
score = cnn4.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.33761082627288996
Test accuracy: 0.9185


In [15]:
## Data Augmentation

In [16]:
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                               height_shift_range=0.08, zoom_range=0.08)
batches = gen.flow(X_train, y_train, batch_size=batch_size)
val_batches = gen.flow(X_val, y_val, batch_size=batch_size)

In [17]:
cnn4.fit_generator(batches, steps_per_epoch=48000//batch_size, epochs=50,
                    validation_data=val_batches, validation_steps=12000//batch_size, use_multiprocessing=True)

Epoch 1/50
93/93 [==============================] - 188s 2s/step - loss: 1.3029 - acc: 0.5049 - val_loss: 0.6583 - val_acc: 0.7386
Epoch 2/50
93/93 [==============================] - 184s 2s/step - loss: 0.6386 - acc: 0.7574 - val_loss: 0.5497 - val_acc: 0.7877
Epoch 3/50
93/93 [==============================] - 187s 2s/step - loss: 0.5300 - acc: 0.7965 - val_loss: 0.4300 - val_acc: 0.8327
Epoch 4/50
93/93 [==============================] - 187s 2s/step - loss: 0.4672 - acc: 0.8231 - val_loss: 0.4123 - val_acc: 0.8404
Epoch 5/50
93/93 [==============================] - 185s 2s/step - loss: 0.4189 - acc: 0.8434 - val_loss: 0.3617 - val_acc: 0.8639
Epoch 6/50
93/93 [==============================] - 186s 2s/step - loss: 0.3821 - acc: 0.8586 - val_loss: 0.3370 - val_acc: 0.8756
Epoch 7/50
93/93 [==============================] - 186s 2s/step - loss: 0.3618 - acc: 0.8667 - val_loss: 0.3230 - val_acc: 0.8806
Epoch 8/50
93/93 [==============================] - 185s 2s/step - loss: 0.3347 - a

In [18]:
score = cnn4.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.22989615490138532
Test accuracy: 0.9268
